# Introduction

Often times, a user may find it more insightful to have confidence intervals together with a point prediction.

Confidence intervals provide a proxy to the range of deviation in the point prediction.

A wider confience interval may signify that the prediction is not very reliable. 

In this notebook, we estimate the confidence interval of a prediction using a quantile objective function.

A quantile objective function can also be used to generate a distribution of the prediction. This may be a source of additional features for downstream tasks.  

log cosh function is used as a smooth approximation to a quantile function. Mathematically, the objective function for the $\alpha$ quantile is given by   

$$
\begin{cases}
  (1-\alpha) \log ( \cosh(x)) & \text{:} & x < 0\\    
  \alpha \log ( \cosh(x)) & \text{:} & x \geq 0    
\end{cases}
$$   

A case study involving the california housing data is also provided.

# Importing packages

In [1]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt

# log cosh quantile function

The problem with using np.cosh function is that there are overflow problems.

We replace the np.cosh definition with an alternate easier to compute definition

In [ ]:
def cosh(x):
    

In [59]:
def log_cosh_quantile(alpha):
    def _log_cosh_quantile(y_true, y_pred):
        err = y_pred - y_true
        err = np.where(err < 0, alpha * err, (1-alpha) * err)
        grad = np.tanh(err)
        #hess = np.where(np.abs(err) < 10, 0, 1 / np.cosh(err)**2)
        hess = sech2(err)
        return grad, hess
    
    def sech2(x):
        #return 0
        #return np.where(np.abs(x) > 100, 0.0, 1.0 / (np.cosh(x)**2) )
        sech = 1 / np.cosh(np.minimum(x, 700 * np.ones(x.shape)))
        
        return sech ** 2
    
    return _log_cosh_quantile

In [45]:
def log_cosh_quantile(alpha):
    def _log_cosh_quantile(y_pred, y_true):
        err = y_pred - y_true
        
        grad = np.where(err < 0, alpha * np.tanh(err), (1-alpha) * np.tanh(err))
        
        hess = np.where(err < 0, alpha * sech2(err), (1-alpha) * sech2(err))
        #hess = err * 0
        
        return grad, hess
    
    def sech2(x):
        #return 0
        #return np.where(np.abs(x) > 100, 0.0, 1.0 / (np.cosh(x)**2) )
        sech = 1 / np.cosh(np.minimum(x, 100 * np.ones(x.shape)))
        
        return sech ** 2
    
    return _log_cosh_quantile

# XGBoost model

In [ ]:
alpha = 0.95
clf = XGBRegressor(objective = log_cosh_quantile(1-alpha),
                  n_entimators = 125,
                  max_depth = 5,
                  n_jobs = 6,
                  learning_rate = 0.05)

# Data loading

In [4]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

In [5]:
splitter = ShuffleSplit(n_splits = 1, test_size = 0.25, random_state = 1)

In [6]:
X = pd.DataFrame(housing.data, columns = housing.feature_names)
y = pd.DataFrame(housing.target, columns = housing.target_names)

# Train test split

In [28]:
for train_index, test_index in splitter.split(X):
    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]

# Training and prediction

In [60]:
alpha = 0.95

clf = XGBRegressor(objective = log_cosh_quantile(alpha),
                  n_estimators = 200,
                  max_depth = 3,
                  n_jobs = 4,
                  learning_rate = 0.05)

In [61]:
clf.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=4, num_parallel_tree=None,
             objective=<function log_cosh_quantile.<locals>._log_cosh_quantile at 0x000001A875C054C0>,
             predictor=None, ...)

In [62]:
y_upper_smooth = clf.predict(X_test)

In [63]:
print(y_upper_smooth[:10])
print(y_test['MedHouseVal'].to_list()[:10])

[4.050434  1.8642776 2.7248468 2.3578942 3.3684528 4.688857  3.0889268
 1.8691504 2.0433846 1.9934542]
[3.55, 0.707, 2.294, 1.125, 2.254, 2.63, 2.268, 1.662, 1.18, 1.563]


In [64]:
clf = XGBRegressor(objective = log_cosh_quantile(1-alpha),
                  n_estimators = 200,
                  max_depth = 3,
                  n_jobs = 4,
                  learning_rate = 0.05)

In [65]:
clf.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=4, num_parallel_tree=None,
             objective=<function log_cosh_quantile.<locals>._log_cosh_quantile at 0x000001A8541679D0>,
             predictor=None, ...)

In [66]:
y_lower_smooth = clf.predict(X_test)

In [67]:
print(y_lower_smooth[:10])
print(y_test['MedHouseVal'].to_list()[:10])

[1.3527011  0.6097829  1.0145602  0.6477446  1.5177529  1.7512562
 1.2843498  0.68816066 0.9595517  0.95071673]
[3.55, 0.707, 2.294, 1.125, 2.254, 2.63, 2.268, 1.662, 1.18, 1.563]
